In [1]:
# this is the download function, it doenload only file that is less than 1 gig and also return the part of the file 

import os,requests
import urllib
import timeit
def download(url):
    max_size = 1000000000
    req = urllib.request.Request(url, method='HEAD')
    feed = urllib.request.urlopen(req)
    size = int(feed.headers['Content-Length'])
    if size < max_size:
        audio_extenions = ['wav', 'mp3']
        get_response = requests.get(url,stream=True)
        file_name  = url.split("/")[-1]
        extension = file_name
        extenstion = file_name.split("/")[-1].split(".")[-1]
        if extenstion in audio_extenions:
            get_response = requests.get(url,stream=True)
            file_name  = url.split("/")[-1]
            with open(file_name, 'wb') as f:
                for chunk in get_response.iter_content(chunk_size=1024):
                    if chunk: # filter out keep-alive new chunks
                        f.write(chunk)
                cwd = os.getcwd()
                print(cwd)

        else:
            print("file not supported")
    else:
        print("over size file")
    
n = 1
result = timeit.timeit(stmt='download("https://relen.s3.us-east-2.amazonaws.com/audio/samples/sample2.wav")', globals=globals(), number=n)
print(f"Execution time is {result / n} seconds")


/Users/macbook/Downloads/testing (5)
Execution time is 18.737971097998525 seconds


In [ ]:
import os

import tensorflow as tf
from tensorflow import keras
# Import packages
from typing import List, Dict
import tqdm.notebook as tq
from tqdm.notebook import tqdm
import json
import pandas as pd
import numpy as np

import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5TokenizerFast as T5Tokenizer
    )

MODEL_NAME = 't5-small'
SOURCE_MAX_TOKEN_LEN = 300
TARGET_MAX_TOKEN_LEN = 80
SEP_TOKEN = '<sep>'

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
tokenizer.add_tokens(SEP_TOKEN)
TOKENIZER_LEN = len(tokenizer)

In [ ]:
class QGModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)
        self.model.resize_token_embeddings(TOKENIZER_LEN) #resizing after adding new tokens to the tokenizer

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return output.loss, output.logits

    def training_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('train_loss', loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('val_loss', loss, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('test_loss', loss, prog_bar=True, logger=True)
        return loss
  
    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=LEARNING_RATE)

In [ ]:
def generate2(qgmodel: QGModel, context: str) -> str:
    source_encoding = tokenizer(
        ' {} {}'.format(SEP_TOKEN, context),
        max_length=SOURCE_MAX_TOKEN_LEN,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )

    generated_ids = qgmodel.model.generate(
        input_ids=source_encoding['input_ids'],
        attention_mask=source_encoding['attention_mask'],
        num_beams=1,
        max_length=TARGET_MAX_TOKEN_LEN,
        repetition_penalty=1.0,
        length_penalty=1.0,
        early_stopping=True,
        use_cache=True
    )

    preds = {
        tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        for generated_id in generated_ids
    }

    return ''.join(preds)



In [ ]:
checkpoint_path = 'multitask-qg-ag.ckpt'

best_model = QGModel.load_from_checkpoint(checkpoint_path)
best_model.freeze()
best_model.eval()

print()

In [ ]:
import nltk.data

In [ ]:
# the function that generate the questions and answer 
def question_generation(text):
    tokenizerzz = nltk.data.load('tokenizers/punkt/english.pickle')
    tope = tokenizerzz.tokenize(text)
    QandA = [] 
    for tee in tope:
        generated = generate2(best_model, tee)
        QandA.append(generated)
    return QandA

n = 1
result = timeit.timeit(stmt='question_generation(text2)', globals=globals(), number=n)
print(f"Execution time is {result / n} seconds")

In [ ]:
# this function seperate the question and answer, checks if the question does not have answer, drop it from then list and print random 3 question 

import random
def clean2(output):
    
    if type(output) == list:
        result = []
        for value in output:
           
            generated = generate2(best_model, value)
            answer = generated.split("<sep>")[0]
            question = generated.split("<sep>")[1]
            if answer == "":
                continue
            dic_word = {"question" : question , "answer" : answer }
            
                     
            result.append(json.dumps(dic_word, indent = 4))
        print(random.choices(result, k = 3))    
            

    else:
        answer = generated.split("<sep>")[0]
        question = generated.split("<sep>")[1]
        dic_word = {"question" : question , "answer" : answer }
        output = json.dumps(dic_word, indent = 2) 
        print(output)

n = 1
result = timeit.timeit(stmt='clean2(QandA)', globals=globals(), number=n)
print(f"Execution time is {result / n} seconds")